In [1]:
# 네이버에서 검색어 입력받아 검색 한 후 블로그 메뉴를 선택하고
# 오른쪽에 있는 검색옵션 버튼을 눌러서
# 정렬 방식과 기간을 입력하기

#Step 0. 필요한 모듈과 라이브러리를 로딩합니다.
import sys # 시스템
import os  # 시스템

import pandas as pd  # 판다스 : 데이터분석 라이브러리
import numpy as np   # 넘파이 : 숫자, 행렬 데이터 라이브러리

from bs4 import BeautifulSoup    # html 데이터를 전처리
from selenium import webdriver   # 웹 브라우저 자동화
from selenium.webdriver.common.keys import Keys
import chromedriver_autoinstaller

import time    # 서버와 통신할 때 중간중간 시간 지연. 보통은 1초
from tqdm import tqdm_notebook   # for문 돌릴 때 진행상황을 %게이지로 알려준다.

# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

In [2]:
# python 버전 확인
!python --version

Python 3.9.7


In [3]:
# 판다스 버전 확인
pd.__version__

'1.3.5'

In [4]:
# !pip install selenium

In [10]:
# 검색어, 검색 기간 정의
query_txt = input('1.크롤링할 키워드는 무엇입니까?: ')

1.크롤링할 키워드는 무엇입니까?: 역삼 맛집


In [6]:
query_txt

'크롤링'

# 1. 크롤링할 블로그 url 수집하기

In [7]:
pwd

'C:\\Users\\korean\\practice'

In [8]:
!pip install selenium

In [11]:
# Step 1. 크롬 웹브라우저 실행
# pip install chromedriver_autoinstaller
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path)

# 사이트 주소는 네이버
driver.get('http://www.naver.com')
time.sleep(2)  # 2초간 정지

In [12]:
# Step 2. 네이버 검색창에 "검색어" 검색
element = driver.find_element_by_id("query")
element.send_keys(query_txt)  # query_txt는 위에서 입력한 키워드
element.submit()
time.sleep(1)

In [14]:
# 첫번째 옷 클릭
CSS_tran="div.Nnq7C.weEfm .eLAPa"                         
driver.find_element_by_css_selector(CSS_tran).click()   # 사진 클릭
time.sleep(1)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.Nnq7C.weEfm .eLAPa"}
  (Session info: chrome=96.0.4664.110)
Stacktrace:
Backtrace:
	Ordinal0 [0x010365C3+2516419]
	Ordinal0 [0x00FCFAA1+2095777]
	Ordinal0 [0x00ED2698+1058456]
	Ordinal0 [0x00EFD2A8+1233576]
	Ordinal0 [0x00EFD49B+1234075]
	Ordinal0 [0x00F27672+1406578]
	Ordinal0 [0x00F1637A+1336186]
	Ordinal0 [0x00F25A1F+1399327]
	Ordinal0 [0x00F1620B+1335819]
	Ordinal0 [0x00EF25E7+1189351]
	Ordinal0 [0x00EF3449+1193033]
	GetHandleVerifier [0x011C3AA4+1573796]
	GetHandleVerifier [0x0126D8E7+2269671]
	GetHandleVerifier [0x010C491B+528923]
	GetHandleVerifier [0x010C3D69+525929]
	Ordinal0 [0x00FD50A9+2117801]
	Ordinal0 [0x00FD9458+2135128]
	Ordinal0 [0x00FD9592+2135442]
	Ordinal0 [0x00FE30F1+2175217]
	BaseThreadInitThunk [0x76BCFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77CE7A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77CE7A6E+238]


In [15]:
# '블로그' 클릭
driver.find_element_by_link_text("블로그").click( )
time.sleep(1)

# '옵션' 클릭
driver.find_element_by_link_text("옵션").click( )
time.sleep(1)

In [16]:
# 검색옵션 확인
item_li = driver.find_elements_by_css_selector('.option .txt')

for i in range(0, len(item_li)):
    print(item_li[i].text)

전체
블로그
카페
관련도순
최신순
전체
1시간
1일
1주
1개월
3개월
6개월
1년
직접입력옵션펼치기


In [17]:
# 검색기간 '3개월' 클릭
print(item_li[10].text)

3개월


In [18]:
item_li[10].click()

In [15]:
# 스크롤 다운
# driver.execute_script("window.scrollTo(0, 500)")
# time.sleep(2)

# 스크롤을 밑으로 내려주는 함수
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0, 19431049)")
    time.sleep(1)

# n: 스크롤할 횟수 설정
n = 3
i = 0
while i < 3: # 이 조건이 만족되는 동안 반복 실행
    scroll_down(driver) # 스크롤 다운
    i = i+1

In [20]:
# 블로그 글 url들 수집
url_list = []
title_list = []

# URL_raw 크롤링 시작
articles = ".api_txt_lines.total_tit"
article_raw = driver.find_elements_by_css_selector(articles)
article_raw[0]

<selenium.webdriver.remote.webelement.WebElement (session="5f16b9e62904c6443855fff856a58abb", element="214d5cee-111f-43b1-9de3-7f3b6a1e79b2")>

In [17]:
article_raw[0].get_attribute('href')

'https://blog.naver.com/ranboy?Redirect=Log&logNo=222576269453'

In [21]:
# 크롤링한 url 정제 시작
for article in article_raw:
    url = article.get_attribute('href')
    url_list.append(url)
time.sleep(1)

# 제목 크롤링 시작
for article in article_raw:
    title = article.text
    title_list.append(title)

    print(title)

print("")
print('url갯수: ', len(url_list))
print('title갯수: ', len(title_list))

무조건인 역삼 맛집 모음
만족스러운 역삼 맛집 리스트
[역삼 맛집] 센터필드 차알 역삼점, 역시나 맛은 실망시키지...
역삼맛집 도야짬뽕 맛있어요
눈과 입이 즐거운 역삼 맛집 리스트
풍요로웠던 역삼 맛집 리스트
중독적인 역삼역 맛집 모음
역삼맛집 소이연남! 점심 뭐먹지? 태국음식 리뷰
[역삼] 파스타맛집 그리고 치즈팩토리 케이크 디스팅트
[역삼 맛집] 정갈한 한식 반상이 있는 한식 다이닝, 수라선
센터필드 맛집 : 역삼 중식당 차알 다녀오다!!!!
역삼 맛집, 세청집(냉살치살)
강남 / 역삼 맛집 _ 핫한 곱도리탕 맛집 : 대낚식당 후기 :D
칭찬하는 역삼역 맛집 베스트
[강남 역삼맛집] 풍천민물장어
[역삼맛집] 쏭타이 - 모든 메뉴가 맛있는 타이음식점
트라가 Traga, 강남 스페인요리 역삼역 맛집(가성비갑)
역삼역 데이트 스페인음식 & 맛집 베스트 5
강남 역삼 맛집 '빌즈'에서 먹은 브런치
[역삼맛집/역삼술집] 맵달 신동궁감자탕 역삼본점
테이크아웃으로 즐기는 킹크랩,랍스타 - 역삼 대게 맛집...
역삼역 맛집, 강남 한정식-봉우리한정식 역삼본점(셰프스폐셜...
역삼 피자 파스타 맛집 지아니스나폴리 역삼에서 점심회식 추천!
반해버린 역삼 맛집 모음 feat 용용선생
회식장소로 제격인 역삼 돼지갈비 맛집
역삼 센터필드 맛집, 퓨전 중식당 차알 Cha'R 센터필드점 방문...
강남/역삼 맛집 _ 디트로이트 피자 : 모터시티바이매니멀
어메이징 역삼 맛집 리스트 5
역삼 꼼장어맛집 박가화로 산꼼장어와 냉삼 강남찐맛집
[강남/역삼 맛집] 유명한 미나리 부대찌개 맛집, 대우부대찌개...
회식장소로 좋은 역삼역 맛집 갈비다움 추천!
[서울 여행/역삼] 연말 송년회... 대게파티 역삼맛집 <묵호항>
역삼 고기집 돼지갈비맛집은 여기!
역삼역 해산물 맛집 역삼회식 비진도해물뚝배기
역삼 센터필드 맛집 js가든 고급 식당에서 중식 먹고 온 후기
역삼 선릉 마라탕 마라샹궈 맛집 한성마라탕 최고!
[서울맛집/역삼 맛집] 강남 스페인요리 맛집 트라가
돼지갈비 맛집 - 갈비삼대 강남역

In [22]:
# 수집된 url들 확인
url_list

['https://blog.naver.com/ranboy?Redirect=Log&logNo=222576269453',
 'https://blog.naver.com/kikijayoung?Redirect=Log&logNo=222526705536',
 'https://blog.naver.com/sh_road?Redirect=Log&logNo=222583401145',
 'https://blog.naver.com/lovely-oh?Redirect=Log&logNo=222607246169',
 'https://blog.naver.com/qufzhd12?Redirect=Log&logNo=222561950718',
 'https://blog.naver.com/mephisto69?Redirect=Log&logNo=222571071136',
 'https://blog.naver.com/zenartist55?Redirect=Log&logNo=222601848412',
 'https://blog.naver.com/kba7628?Redirect=Log&logNo=222600741330',
 'https://blog.naver.com/domhm?Redirect=Log&logNo=222605517774',
 'https://blog.naver.com/dlwodus92?Redirect=Log&logNo=222562510747',
 'https://blog.naver.com/sks5671?Redirect=Log&logNo=222574170262',
 'https://blog.naver.com/yscsm2?Redirect=Log&logNo=222565618730',
 'https://blog.naver.com/naye517?Redirect=Log&logNo=222532835639',
 'https://blog.naver.com/som_a621?Redirect=Log&logNo=222590037073',
 'https://blog.naver.com/nancho58?Redirect=Log&lo

In [23]:
# 수집된 블로그 제목 확인
title_list

['무조건인 역삼 맛집 모음',
 '만족스러운 역삼 맛집 리스트',
 '[역삼 맛집] 센터필드 차알 역삼점, 역시나 맛은 실망시키지...',
 '역삼맛집 도야짬뽕 맛있어요',
 '눈과 입이 즐거운 역삼 맛집 리스트',
 '풍요로웠던 역삼 맛집 리스트',
 '중독적인 역삼역 맛집 모음',
 '역삼맛집 소이연남! 점심 뭐먹지? 태국음식 리뷰',
 '[역삼] 파스타맛집 그리고 치즈팩토리 케이크 디스팅트',
 '[역삼 맛집] 정갈한 한식 반상이 있는 한식 다이닝, 수라선',
 '센터필드 맛집 : 역삼 중식당 차알 다녀오다!!!!',
 '역삼 맛집, 세청집(냉살치살)',
 '강남 / 역삼 맛집 _ 핫한 곱도리탕 맛집 : 대낚식당 후기 :D',
 '칭찬하는 역삼역 맛집 베스트',
 '[강남 역삼맛집] 풍천민물장어',
 '[역삼맛집] 쏭타이 - 모든 메뉴가 맛있는 타이음식점',
 '트라가 Traga, 강남 스페인요리 역삼역 맛집(가성비갑)',
 '역삼역 데이트 스페인음식 & 맛집 베스트 5',
 "강남 역삼 맛집 '빌즈'에서 먹은 브런치",
 '[역삼맛집/역삼술집] 맵달 신동궁감자탕 역삼본점',
 '테이크아웃으로 즐기는 킹크랩,랍스타 - 역삼 대게 맛집...',
 '역삼역 맛집, 강남 한정식-봉우리한정식 역삼본점(셰프스폐셜...',
 '역삼 피자 파스타 맛집 지아니스나폴리 역삼에서 점심회식 추천!',
 '반해버린 역삼 맛집 모음 feat 용용선생',
 '회식장소로 제격인 역삼 돼지갈비 맛집',
 "역삼 센터필드 맛집, 퓨전 중식당 차알 Cha'R 센터필드점 방문...",
 '강남/역삼 맛집 _ 디트로이트 피자 : 모터시티바이매니멀',
 '어메이징 역삼 맛집 리스트 5',
 '역삼 꼼장어맛집 박가화로 산꼼장어와 냉삼 강남찐맛집',
 '[강남/역삼 맛집] 유명한 미나리 부대찌개 맛집, 대우부대찌개...',
 '회식장소로 좋은 역삼역 맛집 갈비다움 추천!',
 '[서울 여행/역삼] 연말 송년회... 대게파티 역삼맛집 <묵호항>',
 '역삼 고기집 돼지갈비맛집은 여기!',
 '역

In [25]:
# 수집된 url_list, title_list로 판다스 데이터프레임 만들기
df = pd.DataFrame({'url':url_list, 'title':title_list})
df

url  \
0    https://blog.naver.com/ranboy?Redirect=Log&log...   
1    https://blog.naver.com/kikijayoung?Redirect=Lo...   
2    https://blog.naver.com/sh_road?Redirect=Log&lo...   
3    https://blog.naver.com/lovely-oh?Redirect=Log&...   
4    https://blog.naver.com/qufzhd12?Redirect=Log&l...   
..                                                 ...   
115  https://blog.naver.com/adias2?Redirect=Log&log...   
116  https://blog.naver.com/chlqudeh123?Redirect=Lo...   
117  https://blog.naver.com/opmiuo?Redirect=Log&log...   
118  https://blog.naver.com/qwer0124?Redirect=Log&l...   
119  https://blog.naver.com/violetdevil3?Redirect=L...   

                                    title  
0                           무조건인 역삼 맛집 모음  
1                         만족스러운 역삼 맛집 리스트  
2    [역삼 맛집] 센터필드 차알 역삼점, 역시나 맛은 실망시키지...  
3                          역삼맛집 도야짬뽕 맛있어요  
4                     눈과 입이 즐거운 역삼 맛집 리스트  
..                                    ...  
115        11. 진가와 / 역삼 진가와 본점 / 우동맛집 진가와  
116             [서울]강남보물섬 역삼역 찐맛집 내돈내산!!!  
117  [역삼 맛집/강남 맛집] 내돈내산 초벌 삼겹살 맛집 연탄마을...  
118                    [역삼] 진대감 - 차돌삼합 맛집  
119                   역삼역 맛집 회식장소로 좋은 진가와  

[120 rows x 2 columns]

In [26]:
pwd

'/Users/aiden/Dropbox/Mac/Desktop/03.crawling'

In [27]:
# 저장하기
df.to_excel("blog_url.xlsx")
# df.to_csv("blog_url.csv")

# 2. 블로그 내용 크롤링하기

In [28]:
import sys
import os
import pandas as pd
import numpy as np

In [33]:
# "url_list.csv" 불러오기
url_load = pd.read_excel("blog_url.xlsx")
url_load = url_load.drop("Unnamed: 0", axis=1)  # 불필요한 칼럼 삭제

num_list = len(url_load)

print(num_list)
url_load

120


url  \
0    https://blog.naver.com/ranboy?Redirect=Log&log...   
1    https://blog.naver.com/kikijayoung?Redirect=Lo...   
2    https://blog.naver.com/sh_road?Redirect=Log&lo...   
3    https://blog.naver.com/lovely-oh?Redirect=Log&...   
4    https://blog.naver.com/qufzhd12?Redirect=Log&l...   
..                                                 ...   
115  https://blog.naver.com/adias2?Redirect=Log&log...   
116  https://blog.naver.com/chlqudeh123?Redirect=Lo...   
117  https://blog.naver.com/opmiuo?Redirect=Log&log...   
118  https://blog.naver.com/qwer0124?Redirect=Log&l...   
119  https://blog.naver.com/violetdevil3?Redirect=L...   

                                    title  
0                           무조건인 역삼 맛집 모음  
1                         만족스러운 역삼 맛집 리스트  
2    [역삼 맛집] 센터필드 차알 역삼점, 역시나 맛은 실망시키지...  
3                          역삼맛집 도야짬뽕 맛있어요  
4                     눈과 입이 즐거운 역삼 맛집 리스트  
..                                    ...  
115        11. 진가와 / 역삼 진가와 본점 / 우동맛집 진가와  
116             [서울]강남보물섬 역삼역 찐맛집 내돈내산!!!  
117  [역삼 맛집/강남 맛집] 내돈내산 초벌 삼겹살 맛집 연탄마을...  
118                    [역삼] 진대감 - 차돌삼합 맛집  
119                   역삼역 맛집 회식장소로 좋은 진가와  

[120 rows x 2 columns]

In [34]:
url_load['url']

0      https://blog.naver.com/ranboy?Redirect=Log&log...
1      https://blog.naver.com/kikijayoung?Redirect=Lo...
2      https://blog.naver.com/sh_road?Redirect=Log&lo...
3      https://blog.naver.com/lovely-oh?Redirect=Log&...
4      https://blog.naver.com/qufzhd12?Redirect=Log&l...
                             ...                        
115    https://blog.naver.com/adias2?Redirect=Log&log...
116    https://blog.naver.com/chlqudeh123?Redirect=Lo...
117    https://blog.naver.com/opmiuo?Redirect=Log&log...
118    https://blog.naver.com/qwer0124?Redirect=Log&l...
119    https://blog.naver.com/violetdevil3?Redirect=L...
Name: url, Length: 120, dtype: object

In [27]:
dict = {}    # 전체 크롤링 데이터를 담을 그릇

# 수집할 글 갯수 정하기
number = 10 

# 수집한 url 돌면서 데이터 수집
for i in tqdm_notebook(range(0, number)):
    # 글 띄우기
    url = url_load['url'][i]
    driver = webdriver.Chrome("/Users/aiden/Desktop/chromedriver")  # 윈도우는 "chromedriver.exe"
    driver.get(url)   # 글 띄우기
    
    # 크롤링
    
    try : 
        # iframe 접근
        driver.switch_to.frame('mainFrame')

        target_info = {}  # 개별 블로그 내용을 담을 딕셔너리 생성

        # 제목 크롤링 시작
        overlays = ".se-module.se-module-text.se-title-text"                                 
        tit = driver.find_element_by_css_selector(overlays)          # title
        title = tit.text

        # 글쓴이 크롤링 시작
        overlays = ".nick"                                 
        nick = driver.find_element_by_css_selector(overlays)         # nickname
        nickname = nick.text

        # 날짜 크롤링
        overlays = ".se_publishDate.pcol2"                                 
        date = driver.find_element_by_css_selector(overlays)         # datetime
        datetime = date.text

        # 내용 크롤링
        overlays = ".se-component.se-text.se-l-default"                                 
        contents = driver.find_elements_by_css_selector(overlays)    # contents

        content_list = []
        for content in contents:
            content_list.append(content.text)
 
        content_str = ' '.join(content_list)                         # content_str

        # 글 하나는 target_info라는 딕셔너리에 담기게 되고,
        target_info['title'] = title
        target_info['nickname'] = nickname
        target_info['datetime'] = datetime
        target_info['content'] = content_str

        # 각각의 글은 dict라는 딕셔너리에 담기게 됩니다.
        dict[i] = target_info
        time.sleep(1)
        
        # 크롤링이 성공하면 글 제목을 출력하게 되고,
        print(i, title)

        # 글 하나 크롤링 후 크롬 창을 닫습니다.
        driver.close()       
    
    # 에러나면 현재 크롬창을 닫고 다음 글(i+1)로 이동합니다.
    except:
        driver.close()
        time.sleep(1)
        continue
    
    # 중간,중간에 파일로 저장하기
    if i == 30 or 50 or 80:
        # 판다스로 만들기
        import pandas as pd
        result_df = pd.DataFrame.from_dict(dict, 'index')

        # 저장하기
        result_df.to_excel("blog_content.xlsx", encoding='utf-8-sig')
        time.sleep(3)

print('수집한 글 갯수: ', len(dict))
print(dict)

  0%|          | 0/10 [00:00<?, ?it/s]

0 무조건인 역삼 맛집 모음 
1 만족스러운 역삼 맛집 리스트
2 [역삼 맛집] 센터필드 차알 역삼점, 역시나 맛은 실망시키지 않는 중식 레스토랑!
3 역삼 중식당 주차가능 맛집 차알
4 눈과 입이 즐거운 역삼 맛집 리스트
5 풍요로웠던 역삼 맛집 리스트
6 반해버린 역삼역 맛집 리스트
7 중독적인 역삼역 맛집 모음
8 역삼맛집 소이연남! 점심 뭐먹지? 태국음식 리뷰
9 [역삼 맛집] 정갈한 한식 반상이 있는 한식 다이닝, 수라선
수집한 글 갯수:  10
{0: {'title': '무조건인 역삼 맛집 모음 ', 'nickname': '어신\n(ranboy)', 'datetime': '2021. 11. 23. 11:30', 'content': '지난 주에 역삼에 사는 친구를 만나러 갔다가 순메밀로 유명한 식당이 있다기에 역삼 맛집을 들려봤어요.\n정갈한 구성의 반찬들과 세계에서 손 꼽히는 진미인 이베리코 베요따를 푸짐하고 먹고온 곳이라 공유해보려 합니다.  주소: 서울 강남구 언주로93길 30 역삼빌딩\n전화번호: 0507-1497-5157\n영업시간: 매일 11:00-21:00 (평일 브레이크 타임 15:00-17:00)\n메뉴: 이베리코 베요따 16,800원 / 막국수 보통 9,500원  이 날 들렸던 식당 백운봉막국수는 코엑스에서 차로 10분 거리였어요.\n건물이 워낙크고, 대로변에 위치해 있어서 단번에 찾을 수 있었답니다.  건물내에는 따로 전용주차장까지 마련되어 있어서 편하게 주차를 할 수 있었어요.\n영업시간은 매일 오전 11시부터 21시까지고, 브레이크타임은 15시부터 17시이니 참고하시면 됩니다.  안으로 들어가니 대체적으로 밝은 인테리어가 눈에 들어왔어요.\n점심시간이 꽤 지난시간이었는데 반대편에는 꽤 많은 손님도 있었답니다.  한쪽에는 셀프바도 세팅되어 있었어요.\n여러명이 이용하는 공간이라 위생이 신경쓰이지 않을 수 없었는데, 상당히 깔끔하게 관리하고 계셨어요! 한쪽에는 샐러드종류부터 곁들임채소까지 준비되어 있었는데요.\n신선도도 뛰어나고 종

In [28]:
dict

{0: {'title': '무조건인 역삼 맛집 모음 ',
  'nickname': '어신\n(ranboy)',
  'datetime': '2021. 11. 23. 11:30',
  'content': '지난 주에 역삼에 사는 친구를 만나러 갔다가 순메밀로 유명한 식당이 있다기에 역삼 맛집을 들려봤어요.\n정갈한 구성의 반찬들과 세계에서 손 꼽히는 진미인 이베리코 베요따를 푸짐하고 먹고온 곳이라 공유해보려 합니다.  주소: 서울 강남구 언주로93길 30 역삼빌딩\n전화번호: 0507-1497-5157\n영업시간: 매일 11:00-21:00 (평일 브레이크 타임 15:00-17:00)\n메뉴: 이베리코 베요따 16,800원 / 막국수 보통 9,500원  이 날 들렸던 식당 백운봉막국수는 코엑스에서 차로 10분 거리였어요.\n건물이 워낙크고, 대로변에 위치해 있어서 단번에 찾을 수 있었답니다.  건물내에는 따로 전용주차장까지 마련되어 있어서 편하게 주차를 할 수 있었어요.\n영업시간은 매일 오전 11시부터 21시까지고, 브레이크타임은 15시부터 17시이니 참고하시면 됩니다.  안으로 들어가니 대체적으로 밝은 인테리어가 눈에 들어왔어요.\n점심시간이 꽤 지난시간이었는데 반대편에는 꽤 많은 손님도 있었답니다.  한쪽에는 셀프바도 세팅되어 있었어요.\n여러명이 이용하는 공간이라 위생이 신경쓰이지 않을 수 없었는데, 상당히 깔끔하게 관리하고 계셨어요! 한쪽에는 샐러드종류부터 곁들임채소까지 준비되어 있었는데요.\n신선도도 뛰어나고 종류도 다양하니 손님을 배려하는 센스가 돋보였죠. 넓은 좌석에 착석 후 메뉴판을 살펴봤어요.\n역삼 맛집에서는 막국수도 유명하지만 최고등급의 이베리코베요따도 취급하고 있어서 주문해봤어요.  주문과 동시에 상차림이 빠르게 이루어졌어요.\n로봇을 이용해 기본 상차림이 이루어져서 신기한 구경도 할 수 있었네요! 먼저 입맛을 올려줄 메밀쌈에 각종 채소를 싸서 입 안에 넣어봤어요. \n최대한 얇게 부쳐주셔서 퍽퍽함이라곤 찾아볼 수 없이 쫄깃했어요.  이어서

In [29]:
# 판다스로 만들기
import pandas as pd
result_df = pd.DataFrame.from_dict(dict, 'index')
result_df

title            nickname  \
0                                 무조건인 역삼 맛집 모음         어신\n(ranboy)   
1                                만족스러운 역삼 맛집 리스트  김새침\n(kikijayoung)   
2  [역삼 맛집] 센터필드 차알 역삼점, 역시나 맛은 실망시키지 않는 중식 레스토랑!  여행하지마리오\n(sh_road)   
3                              역삼 중식당 주차가능 맛집 차알                  슈슈   
4                            눈과 입이 즐거운 역삼 맛집 리스트                두덩두덩   
5                                풍요로웠던 역삼 맛집 리스트    느느\n(mephisto69)   
6                                반해버린 역삼역 맛집 리스트                  퍼만   
7                                 중독적인 역삼역 맛집 모음                먹방토끼   
8                     역삼맛집 소이연남! 점심 뭐먹지? 태국음식 리뷰      델피늄\n(kba7628)   
9              [역삼 맛집] 정갈한 한식 반상이 있는 한식 다이닝, 수라선                  템포   

              datetime                                            content  
0  2021. 11. 23. 11:30  지난 주에 역삼에 사는 친구를 만나러 갔다가 순메밀로 유명한 식당이 있다기에 역삼 ...  
1    2021. 10. 5. 9:10  안녕 김새침언니에요.\n예전에 살던 동네를 마주치게 되면\n익숙한 골목골목 지나면서...  
2   2021. 12. 1. 12:00  지난주 월요일 여자친구와 예비 처제, 동서와 4명이서 센터필드 맛집인 '차알' 역삼...  
3   2021. 12. 3. 23:36  역삼 중식당 맛집 차알에서\n너무 만족스러운 식사하고 와서\n포스팅 써봐요! 역삼 ...  
4    2021. 11. 9. 9:00  안녕하세요. 두덩이에요 :)\n오늘은 역삼 맛집들을 포스팅할까 합니다!\n그중에서도...  
5  2021. 11. 22. 13:00  지난 주에 친구와 점심 약속을 잡고 역삼 맛집에 다녀왔는데요. 덮밥을 수북히 쌓아주...  
6  2021. 12. 15. 13:06  몇주전 친한 지인들과 강남쪽에 드라이브갔다가 SNS에서 유명한 역삼역 맛집에 다녀왔...  
7  2021. 12. 23. 13:00  며칠 전 친구와 쇼핑을 나갔다가 손님이 바글바글한 역삼역 맛집에 방문하게 됐어요. ...  
8  2021. 12. 21. 10:44  http://naver.me/GgekT3fA 아침부터 쌀쌀한 날씨에\n따끈한 국물이...  
9   2021. 11. 13. 8:50  흔히 말하는 집 밥에서부터\n다양한 메뉴들이 나오는 고급 진 한정식까지\n한식의 종...

In [30]:
# 엑셀로 저장하기
result_df.to_excel("blog_content.xlsx", encoding='utf-8-sig')

In [26]:
pwd

'/Users/aiden/Dropbox/Mac/Desktop/크롤러'